In [1]:
from hypex.dataset.dataset import Dataset, ExperimentData
from hypex.dataset.roles import (
    InfoRole,
    FeatureRole,
    TreatmentRole,
    PreTargetRole,
    TargetRole,
)
from hypex.experiments.aa_test import AA_TEST

In [3]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": PreTargetRole(),
        "post_spends": TargetRole(),
        "age": FeatureRole(),
        "gender": FeatureRole(),
        "industry": FeatureRole(),
    }, data="data.csv",
)
data

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [3]:
data.roles

{'user_id': <hypex.dataset.roles.InfoRole at 0x204b1975450>,
 'signup_month': <hypex.dataset.roles.FeatureRole at 0x204b1975990>,
 'treat': <hypex.dataset.roles.TreatmentRole at 0x204b1975000>,
 'pre_spends': <hypex.dataset.roles.PreTargetRole at 0x204b1975060>,
 'post_spends': <hypex.dataset.roles.TargetRole at 0x204b19751b0>,
 'age': <hypex.dataset.roles.FeatureRole at 0x204b19750f0>,
 'gender': <hypex.dataset.roles.FeatureRole at 0x204b1974d30>,
 'industry': <hypex.dataset.roles.FeatureRole at 0x204b1974b50>}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

c:\Projects\HypEx\hypex\dataset\dataset.py:55: UserWarning: Didn't add roles for experiment estimation. This fact can lead to incorrect operation of the pipeline. 
  warnings.warn(
c:\Projects\HypEx\hypex\dataset\dataset.py:55: UserWarning: Didn't add roles for experiment estimation. This fact can lead to incorrect operation of the pipeline. 
  warnings.warn(
c:\Projects\HypEx\hypex\dataset\dataset.py:55: UserWarning: Didn't add roles for experiment estimation. This fact can lead to incorrect operation of the pipeline. 
  warnings.warn(
c:\Projects\HypEx\hypex\experiment\experiment.py:223: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data.data.tmp_roles = {field: TempTargetRole}


roles {'user_id': <hypex.dataset.roles.InfoRole object at 0x00000204B1975450>, 'signup_month': <hypex.dataset.roles.FeatureRole object at 0x00000204B1975990>, 'treat': <hypex.dataset.roles.TreatmentRole object at 0x00000204B1975000>, 'pre_spends': <hypex.dataset.roles.PreTargetRole object at 0x00000204B1975060>, 'post_spends': <hypex.dataset.roles.TargetRole object at 0x00000204B19751B0>, 'age': <hypex.dataset.roles.FeatureRole object at 0x00000204B19750F0>, 'gender': <hypex.dataset.roles.FeatureRole object at 0x00000204B1974D30>, 'industry': <hypex.dataset.roles.FeatureRole object at 0x00000204B1974B50>}
['post_spends']
roles {'user_id': <hypex.dataset.roles.InfoRole object at 0x00000204B1975450>, 'signup_month': <hypex.dataset.roles.FeatureRole object at 0x00000204B1975990>, 'treat': <hypex.dataset.roles.TreatmentRole object at 0x00000204B1975000>, 'pre_spends': <hypex.dataset.roles.PreTargetRole object at 0x00000204B1975060>, 'post_spends': <hypex.dataset.roles.TargetRole object at 

IndexError: list index out of range

In [ ]:
ed.data.data.sample(frac=1, random_state=1)

AttributeError: 'DataFrame' object has no attribute 'data'

In [ ]:
control_std = control_group.std()
test_std = test_group.std()

test_proportion = len(test_group) / (len(test_group) + len(control_group))
control_proportion = 1 - test_proportion

d = ((norm.ppf(1 - significance / 2) + norm.ppf(power)) / mde) ** 2
s = test_std ** 2 / test_proportion + control_std ** 2 / control_proportion
return d * s